In [1]:
import tensorflow as tf
import os
import shutil

import base64
import json
from oauth2client.client import GoogleCredentials
import requests

In [2]:
CHECK_POINT_DIR='gs://fire_detection_anurag/models/accuracy/chkpts'

model = tf.keras.models.load_model(CHECK_POINT_DIR)
print(model.summary())

Model: "fire_detection"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random/center_crop (RandomCr (None, 224, 224, 3)       0         
_________________________________________________________________
random_lr_flip (RandomFlip)  (None, 224, 224, 3)       0         
_________________________________________________________________
random_contrast_brightness/n (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenet_embedding (KerasLa (None, 1280)              2257984   
_________________________________________________________________
dense_hidden_1 (Dense)       (None, 32)                40992     
_________________________________________________________________
dense_hidden_2 (Dense)       (None, 16)                528       
_________________________________________________________________
fire_detector (Dense)        (None, 2)              

In [3]:
MODEL_LOCATION = 'export/fire_bytes_model'  # will be created

IMG_HEIGHT = 448  # same value as used while model training
IMG_WIDTH = IMG_HEIGHT
IMG_CHANNELS = 3
CLASS_NAMES = 'Fire No-Fire'.split()

Check manually first

In [4]:
def read_from_jpegfile(filename):
    img_bytes = tf.io.read_file(filename)
    return img_bytes
    
def preprocess(img_bytes):
    img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize_with_pad(img, IMG_HEIGHT, IMG_WIDTH)

In [5]:
filenames = tf.io.gfile.glob("gs://fire_detection_anurag/test_images/*")

filenames

['gs://fire_detection_anurag/test_images/fire1.jpg',
 'gs://fire_detection_anurag/test_images/fire2.jpg',
 'gs://fire_detection_anurag/test_images/fire3.jpg',
 'gs://fire_detection_anurag/test_images/fire4.jpg',
 'gs://fire_detection_anurag/test_images/fire5.jpg',
 'gs://fire_detection_anurag/test_images/no_fire1.jpg',
 'gs://fire_detection_anurag/test_images/no_fire2.jpg',
 'gs://fire_detection_anurag/test_images/no_fire3.jpg',
 'gs://fire_detection_anurag/test_images/no_fire4.jpg',
 'gs://fire_detection_anurag/test_images/no_fire5.jpg']

In [6]:
for filename in filenames:
    img_bytes = read_from_jpegfile(filename)
    img = preprocess(img_bytes)
    img = tf.expand_dims(img, axis=0)
    pred = model.predict(img)
    print(pred)

[[0.9867954  0.01320453]]
[[0.71165985 0.28834015]]
[[0.99001807 0.0099819 ]]
[[0.9839685  0.01603149]]
[[0.9958753  0.00412475]]
[[0.17246704 0.82753295]]
[[0.0099609  0.99003905]]
[[0.01525044 0.98474956]]
[[0.01463307 0.98536694]]
[[0.00689968 0.9931003 ]]


## Set model signature which can predict on image bytes coming directly i.e. without image being first loaded to GCS

In [7]:
# set model signature-1 (to be used as default)
@tf.function(input_signature=[tf.TensorSpec([None,],
                                            dtype=tf.string)])
def predict_bytes(img_bytes):
    input_images = tf.map_fn(
                             preprocess,
                             img_bytes,
                             fn_output_signature=tf.float32
                            )
    batch_pred = model(input_images) # same as model.predict()
    top_prob = tf.math.reduce_max(batch_pred,
                                  axis=[1])
    pred_label_index = tf.math.argmax(batch_pred,
                                      axis=1)
    pred_label = tf.gather(tf.convert_to_tensor(CLASS_NAMES),
                           pred_label_index)
    
    return {
            'probability': top_prob,
            'image_type_int': pred_label_index,
            'image_type_str': pred_label
           }


# set model signature-2
# If image file is provided, we first need to convert it to bytes
@tf.function(input_signature=[tf.TensorSpec([None,],
                                            dtype=tf.string)])
def predict_filename(filenames):
    img_bytes = tf.map_fn(
                          tf.io.read_file,  # extract bytes
                          filenames
                         )
    result = predict_bytes(img_bytes)  # get prediction
    result['filename'] = filenames  # map with corresponding input image
    
    return result

In [8]:
shutil.rmtree('export',
              ignore_errors=True)
os.mkdir('export')

model.save(MODEL_LOCATION,
           signatures={
                       'serving_default': predict_bytes,
                       'from_images': predict_filename
                      }
          )

INFO:tensorflow:Assets written to: export/fire_bytes_model/assets


In [9]:
!saved_model_cli show --tag_set serve --dir $MODEL_LOCATION

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "from_images"
SignatureDef key: "serving_default"


In [10]:
!saved_model_cli show --tag_set serve --dir $MODEL_LOCATION --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['img_bytes'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_img_bytes:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['image_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall_1:0
  outputs['image_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall_1:1
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall_1:2
Method name is: tensorflow/serving/predict


In [11]:
!saved_model_cli show --tag_set serve --dir $MODEL_LOCATION --signature_def from_images

The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: from_images_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['filename'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['image_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall:1
  outputs['image_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:2
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall:3
Method name is: tensorflow/serving/predict


Check locally

In [12]:
filenames[0]

'gs://fire_detection_anurag/test_images/fire1.jpg'

In [13]:
!gsutil cp gs://fire_detection_anurag/test_images/fire1.jpg /tmp/fire1.jpg

Copying gs://fire_detection_anurag/test_images/fire1.jpg...
/ [1 files][ 26.5 KiB/ 26.5 KiB]                                                
Operation completed over 1 objects/26.5 KiB.                                     


In [15]:
with open('/tmp/fire1.jpg', 'rb') as ifp:
    img_bytes = ifp.read()
    serving_fn = tf.keras.models.load_model(MODEL_LOCATION).signatures['serving_default']
    pred = serving_fn(tf.convert_to_tensor([img_bytes]))
    print(pred)

{'image_type_int': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>, 'image_type_str': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Fire'], dtype=object)>, 'probability': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.99452174], dtype=float32)>}


## Deploy model to Vertex-AI endpoint

In [16]:
!gsutil -m cp -r export/* gs://fire_detection_anurag/models/bytes_inferencing/export/

Copying file://export/fire_bytes_model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://export/fire_bytes_model/keras_metadata.pb [Content-Type=application/octet-stream]...
Copying file://export/fire_bytes_model/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://export/fire_bytes_model/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
/ [4/4 files][ 10.9 MiB/ 10.9 MiB] 100% Done                                    
Operation completed over 4 objects/10.9 MiB.                                     


In [17]:
%%writefile vertex_ai_bytes_model_deploy.sh
REGION="us-central1"
ENDPOINT_NAME="fire_bytes_endpoint"
MODEL_NAME="fire_detector_bytes"
MODEL_LOCATION="gs://fire_detection_anurag/models/bytes_inferencing/export/fire_bytes_model"
IMAGE_URI="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-5:latest"

for i in "$@"
do
case $i in
        -r=*|--region=*) REGION="${i#*=}"; shift ;;
        -e=*|--endpoint_name=*) ENDPOINT_NAME="${i#*=}"; shift ;;
        -m=*|--model_name=*) MODEL_NAME="${i#*=}"; shift ;;
        -l=*|--model_location=*) MODEL_LOCATION="${i#*=}"; shift ;;
        -i=*|--image_uri=*) IMAGE_URI="${i#*=}"; shift ;;
        *) echo "Unknown parameter passed: $1"; exit 1 ;;
esac
done

echo "Deploying model $MODEL_NAME"

if [[ $(gcloud ai endpoints list --region=$REGION --format="value(display_name)" | grep $ENDPOINT_NAME) ]]; then
    echo "The endpoint named $ENDPOINT_NAME already exists."
else
    # Create endpoint.
    echo "Creating $ENDPOINT_NAME endpoint now."
    gcloud ai endpoints create \
      --region=$REGION \
      --display-name=$ENDPOINT_NAME
fi

ENDPOINT_ID=$(gcloud ai endpoints list --region=$REGION --format="value(name)" --filter="displayName=$ENDPOINT_NAME")
echo "The endpoint_id is $ENDPOINT_ID"

if [[ $(gcloud ai models list --region=$REGION --format="value(display_name)" | grep $MODEL_NAME) ]]; then
    echo "The model named $MODEL_NAME already exists."
else
    # Upload model.
    echo "Uploading $MODEL_NAME model now."
    gcloud ai models upload \
      --region=$REGION \
      --display-name=$MODEL_NAME \
      --container-image-uri=$IMAGE_URI \
      --artifact-uri=$MODEL_LOCATION
fi

MODEL_ID=$(gcloud ai models list --region=$REGION --format="value(name)" --filter="displayName=$MODEL_NAME")
echo "The model_id is $MODEL_ID"

echo "Deploying model now"
gcloud ai endpoints deploy-model $ENDPOINT_ID\
  --region=$REGION \
  --model=$MODEL_ID \
  --display-name=$MODEL_NAME \
  --traffic-split=0=100

Overwriting vertex_ai_bytes_model_deploy.sh


In [18]:
!bash vertex_ai_bytes_model_deploy.sh

Deploying model fire_detector_bytes
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Creating fire_bytes_endpoint endpoint now.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [8453295052200869888]...done.                            
Created AI Platform endpoint: projects/9118975290/locations/us-central1/endpoints/4876703505093492736.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
The endpoint_id is 4876703505093492736
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Uploading fire_detector_bytes model now.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [1504240827168194560]...done.                            
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
The model_id is projects/9118975290/locations/us-central1/models/1620526207713935360
Deploying model now
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [64

In [19]:
PROJECT = "kubeflow-1-0-2"
REGION = "us-central1"
ENDPOINT_ID = "4876703505093492736"

Helper function

In [20]:
def b64encode(filename):
    with open(filename, 'rb') as ifp:
        img_bytes = ifp.read()
        return base64.b64encode(img_bytes)

In [21]:
token = GoogleCredentials.get_application_default().get_access_token().access_token

api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/endpoints/{}:predict".format(
                                                                                                     REGION,
                                                                                                     PROJECT,
                                                                                                     REGION,
                                                                                                     ENDPOINT_ID
                                                                                                    )

headers = {"Authorization": "Bearer " + token }

In [22]:
api

'https://us-central1-aiplatform.googleapis.com/v1/projects/kubeflow-1-0-2/locations/us-central1/endpoints/4876703505093492736:predict'

In [23]:
filenames

['gs://fire_detection_anurag/test_images/fire1.jpg',
 'gs://fire_detection_anurag/test_images/fire2.jpg',
 'gs://fire_detection_anurag/test_images/fire3.jpg',
 'gs://fire_detection_anurag/test_images/fire4.jpg',
 'gs://fire_detection_anurag/test_images/fire5.jpg',
 'gs://fire_detection_anurag/test_images/no_fire1.jpg',
 'gs://fire_detection_anurag/test_images/no_fire2.jpg',
 'gs://fire_detection_anurag/test_images/no_fire3.jpg',
 'gs://fire_detection_anurag/test_images/no_fire4.jpg',
 'gs://fire_detection_anurag/test_images/no_fire5.jpg']

In [24]:
%%bash
gsutil cp gs://fire_detection_anurag/test_images/fire1.jpg /tmp/fire1.jpg
gsutil cp gs://fire_detection_anurag/test_images/fire2.jpg /tmp/fire2.jpg
gsutil cp gs://fire_detection_anurag/test_images/no_fire1.jpg /tmp/no_fire1.jpg
gsutil cp gs://fire_detection_anurag/test_images/no_fire2.jpg /tmp/no_fire2.jpg
gsutil cp gs://fire_detection_anurag/test_images/no_fire3.jpg /tmp/no_fire3.jpg

Copying gs://fire_detection_anurag/test_images/fire1.jpg...
/ [1 files][ 26.5 KiB/ 26.5 KiB]                                                
Operation completed over 1 objects/26.5 KiB.                                     
Copying gs://fire_detection_anurag/test_images/fire2.jpg...
/ [1 files][ 83.0 KiB/ 83.0 KiB]                                                
Operation completed over 1 objects/83.0 KiB.                                     
Copying gs://fire_detection_anurag/test_images/no_fire1.jpg...
/ [1 files][  7.0 KiB/  7.0 KiB]                                                
Operation completed over 1 objects/7.0 KiB.                                      
Copying gs://fire_detection_anurag/test_images/no_fire2.jpg...
/ [1 files][  2.6 MiB/  2.6 MiB]                                                
Operation completed over 1 objects/2.6 MiB.                                      
Copying gs://fire_detection_anurag/test_images/no_fire3.jpg...
/ [1 files][  1.4 MiB/  1.4 MiB]       

**Note**: Since jpg files might contain special characters, it's better to encode (base64) the file content, before sending the request.

In [45]:
data = {
        "instances": [
                      {
                       "img_bytes": {"b64": b64encode('/tmp/fire1.jpg')}
                      },
                      {
                       "img_bytes": {"b64": b64encode('/tmp/no_fire1.jpg')}
                      },
                     ]
       }

In [46]:
response = requests.post(api,
                         json=data,
                         headers=headers)

In [47]:
print(response.content)

b'{\n  "predictions": [\n    {\n      "image_type_int": 0,\n      "probability": 0.994521737,\n      "image_type_str": "Fire"\n    },\n    {\n      "probability": 0.827533,\n      "image_type_int": 1,\n      "image_type_str": "No-Fire"\n    }\n  ],\n  "deployedModelId": "924830016447971328",\n  "model": "projects/9118975290/locations/us-central1/models/1620526207713935360",\n  "modelDisplayName": "fire_detector_bytes"\n}\n'


In [48]:
json.loads(response.content.decode('utf-8').replace('\n', ''))

{'predictions': [{'image_type_int': 0,
   'probability': 0.994521737,
   'image_type_str': 'Fire'},
  {'probability': 0.827533, 'image_type_int': 1, 'image_type_str': 'No-Fire'}],
 'deployedModelId': '924830016447971328',
 'model': 'projects/9118975290/locations/us-central1/models/1620526207713935360',
 'modelDisplayName': 'fire_detector_bytes'}

**Note**: If we encounter a limit on the size of bytes sent at a time (due to large no. of prediction requests), that can hinder inferencing at scale. In that case, we can try parallelizing the inferencing process e.g. using Apache Beam, like we did in one of the notebooks for inferencing from image files.